** DNN Using Tensorflow **

This notebook uses tensorflow's python api to create a deep neural network

## Construction Phase

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

import numpy as np
import tensorflow as tf

# Defining variables for MNIST dataset
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

Create Network with 2 hidden layers

In [2]:
# Creating input and output shapes
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

### Define Network

#### Create network by defining our own layers

In [3]:
# Create a layer
def neuron_layer(X, n_neurons, name, activation=None):
    # Create a name scope for each layer
    with tf.name_scope(name):
        # Get input shape
        n_inputs = int(X.get_shape()[1])
        # Create weights vector
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        # Create bias vector
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        # Generate output
        z = tf.matmul(X, W) + b
        # Use relu activation if specified
        if activation=="relu":
            return tf.nn.relu(z)
        else:
            return z

In [4]:
# Create a name scope for the network
with tf.name_scope("dnn"):
    # create a hidden layer with inputs from the input layer
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    # create a hidden layer with inputs from the first hidden layer
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    # create an output layer with inputs from the second hidden layer
    logits  = neuron_layer(hidden2, n_outputs, "outputs")

#### Create a network by using tensorflow's layers

In [5]:
from tensorflow.contrib.layers import fully_connected
# Create a name scope for the network
with tf.name_scope("dnn"):
    # create a hidden layer with inputs from the input layer
    hidden1 = fully_connected(X, n_hidden1, scope="hidden1")
    # create a hidden layer with inputs from the first hidden layer
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")
    # create an output layer with inputs from the second hidden layer
    logits  = fully_connected(hidden2, n_outputs, scope="outputs", activation_fn=None)

### Define Cost Function

In [6]:
# Create a name scope for the cost function
with tf.name_scope("loss"):
    # Apply the softmax and then evaluate cross-entropy
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

### Define Optimizer

In [7]:
learning_rate = 0.01

# Create a name scope for the optimizer
with tf.name_scope("train"):
    # Use gradient descent for the optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

### Define Evaluation

In [8]:
# Create a name scope for the evaluation
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

### Final Steps

In [9]:
# Create a node to initialize all variables
init = tf.global_variables_initializer()

# Create a saver to save our trained model
saver = tf.train.Saver()

## Execution Phase

In [10]:
# Load MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [11]:
# Define hyperparameters
n_epochs   = 20
batch_size = 50

**Train the model**

### Creating a new session

In [12]:
# Open Tensorflow session
with tf.Session() as sess:
    # Initialize variables
    init.run()
    # Run through main training loop
    for epoch in range(n_epochs):
        # Run through each batch
        for iteration in range(mnist.train.num_examples // batch_size):
            # Get the next batch
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            # Run through the batch
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        # Evaluate training accuracy
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        # Evaluate testing accuracy
        acc_test  = accuracy.eval(feed_dict={X: mnist.test.images, 
                                             y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test_accuracy", acc_test)
    # Save the model
    save_path = saver.save(sess, "./dnn_model.ckpt")

0 Train accuracy: 0.96 Test_accuracy 0.8973
1 Train accuracy: 0.98 Test_accuracy 0.9215
2 Train accuracy: 0.98 Test_accuracy 0.9346
3 Train accuracy: 1.0 Test_accuracy 0.9381
4 Train accuracy: 0.92 Test_accuracy 0.9429
5 Train accuracy: 0.92 Test_accuracy 0.9487
6 Train accuracy: 0.98 Test_accuracy 0.9506
7 Train accuracy: 0.98 Test_accuracy 0.9546
8 Train accuracy: 0.96 Test_accuracy 0.958
9 Train accuracy: 0.98 Test_accuracy 0.9588
10 Train accuracy: 0.92 Test_accuracy 0.9608
11 Train accuracy: 0.98 Test_accuracy 0.963
12 Train accuracy: 0.98 Test_accuracy 0.9638
13 Train accuracy: 0.98 Test_accuracy 0.9652
14 Train accuracy: 0.98 Test_accuracy 0.9659
15 Train accuracy: 1.0 Test_accuracy 0.9672
16 Train accuracy: 0.98 Test_accuracy 0.967
17 Train accuracy: 1.0 Test_accuracy 0.9687
18 Train accuracy: 0.96 Test_accuracy 0.9707
19 Train accuracy: 0.98 Test_accuracy 0.9709


### Loading a previous session

In [13]:
# Open Tensorflow session
with tf.Session() as sess:
    # Restore the model
    saver.restore(sess, "./dnn_model.ckpt")
    # Get new set of data
    X_new_scaled = [....] # A new set of images to evaluate, must be scaled from 0 to 1
    # Evaluate new dataset on the model
    Z = logits.eval(feed_dict={X: X_new_scaled})
    # Generate the prediction
    y_pred = np.argmax(Z, axis=1)

SyntaxError: invalid syntax (<ipython-input-13-6f27a71e7190>, line 6)